In [2]:
import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from pykrx import stock
from pykrx import bond
from io import StringIO

def get_csv_finance(fromDate, toDate, ticker):
    # dataframe 생성
    df = stock.get_market_ohlcv(fromDate, toDate, ticker)

    csv_buffer = StringIO()
    # dataframe -> csv 형태로 변환
    df.to_csv(csv_buffer)
    return csv_buffer.getvalue()

In [ ]:
# 삼성 : 005930
# 할루시네이션 방지용으로 티커 코드는 제공해주는 것
messages = [{"role": "user", "content": "삼성전자(005930) 2022년 8월 1일부터 2022년 12월 1일까지의 데이터를 분석해서 보고해줘."}]
functions = [
    {
        "name": "get_csv_finance",
        "discription": "날짜별 삼성전자 주가를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "fromDate": {"type": "string", "description": "시작 일자"},
                "toDate": {"type": "string", "description": "종료 일자"},
                "ticker": {"type": "string", "description":"6자리 숫자 형식의 종목 티커코드"}
            },
            "required": ["fromDate", "toDate", "ticker"]
        }
    }
]


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto",
)

response_message = response.choices[0].message
print(response_message)


In [ ]:
# 데이터를 가져와서 실행

if response_message.function_call is not None:
    available_functions = {
        "get_csv_finance" : get_csv_finance,
    }

    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        fromDate=function_args['fromDate'],
        toDate=function_args['toDate'],
        ticker=function_args['ticker']
    )
    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response.choices[0].message.content)